# Unstack to Bokeh Bar Chart
Replaying Mrs. Sarah Bird's introductory [code snippets to Bokeh](https://github.com/birdsarah/europython-2015-bokeh/blob/master/static/slides.md) from the EuroPython 2015 conference, I stumbled upon the shenanigans of Bokeh having "obliterated" the Chart interface in version [0.11](http://bokeh.pydata.org/en/latest/docs/releases/0.11.0.html) ;-)
In Bokeh 0.10, you could resort to the bokeh._legacy_charts module.

Well, [Chart](http://bokeh.pydata.org/en/0.11.0/docs/reference/charts.html#bar) doesn't have the stacked=True parameter, any more. Instead, there is the stack parameter that expects a list of categories to stack upon that go side by side the list of values and the labels. Normally, you'd have the data to pass in. But there might be cases where you don't want to look into changing your code. So, I've thrown in two cents of a short method called [unstack_to_bar](https://gist.github.com/kgrgnff/e4c77bb4d3c48ffd96e2) that chains, zips and cycles to also work when there are more two categories in the summarized form, already.

In [1]:
import pandas as pd
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [42]:
# sample from original
# https://raw.githubusercontent.com/birdsarah/europython-2015-bokeh/master/notebooks/Chart.ipynb

#from process_gtimelog import get_bar_chart_df
#bar_df = get_bar_chart_df()
#bar_df.head()

# mock bar_df results of the example without gtimelog processing ...
mydata = {'sub_activity' : ['General', 'Mailing List'],
            'human': [7.88, 0.22],
            'automated': [0.22, 7.88]}
bar_df = pd.DataFrame(mydata, columns=['sub_activity', 'human', 'automated'])
bar_df = bar_df.set_index('sub_activity')
bar_df

,human,automated
sub_activity,,
General,7.88,0.22
Mailing List,0.22,7.88


In [46]:
from bokeh.charts import Bar
#show(Bar(bar_df, stacked=True))

##Yields AttributeError: unexpected attribute 'stacked' to Chart, possible attributes are above, background_fill_alpha, backg

In [38]:
# fix missing stacked=True
import itertools
def unstack_to_bar(df):
    """
    unwind the stacked entries of df_matrix
    
    return dictionary with lists for values, label, and stack 
    to be passed on to bokeh.charts.Bar
    """
    values = list(df.unstack())
    
    label = df.index.tolist()*len(df.columns)
    
    header = df.columns.tolist()
    stack = list(itertools.chain.from_iterable(zip(header, itertools.cycle(header))))
    
    bar_dict = {}
    bar_dict.update({'values': values, 'label': label, 'stack': stack})
    
    return bar_dict

In [43]:
data = unstack_to_bar(bar_df)
data
show(Bar(data, values='values', label='label', stack='stack',
    title="Stacked Bar Chart Demo", legend='top_right', xlabel='Sub Activity', ylabel = '', 
    palette=['gray', 'green']))

In [44]:
# test with two more purely fictive useless categories
bar_df['RESTful service'] = [.5, 1]
bar_df['Rhodaniversum'] = [1, .5]
bar_df

,human,automated,RESTful service,Rhodaniversum
sub_activity,,,,
General,7.88,0.22,0.5,1.0
Mailing List,0.22,7.88,1.0,0.5


In [49]:
from bokeh.palettes import Spectral6
data = unstack_to_bar(bar_df)
show(Bar(data, values='values', label='label', stack='stack',
    title="Stacked Bar Chart Demo 2", legend='top_right', xlabel='Sub Activity', ylabel = '', 
    palette=Spectral6))

Output: 
![stacked bar chart demo 2](./demo2.png "Demo")